In [1]:
import numpy as np
import pandas as pd
import torch
import os
from IPython.display import clear_output
import matplotlib.pyplot as plt
torch.__version__

'2.2.2'

# Code to handle to create and handle the dataset

In [2]:
from models.DGDNN.Data.geometric_dataset_gen import MyDataset as MyGeometricDataset
from models.DGDNN.Data.dataset_gen import MyDataset 

In [3]:
%reload_ext autoreload
%autoreload 2

In [9]:
dir = 'data/datasets/America_Stocks_reduced'
dest = 'data/datasets/graph/'
market = 'NASDAQ'
company_list = ['AAPL', 'AMZN', 'AMRK', 'APOG']
start_date = '2010-01-01'
end_date = '2015-12-31'
window_size = 30  # 30 days window
dataset_label = 'train'
use_fast_approximation = False

# Create a dataset instance
dataset = MyGeometricDataset(
    root=dir,
    desti=dest,
    market=market,
    comlist=company_list,
    start=start_date,
    end=end_date,
    window=window_size,
    dataset_type=dataset_label,
    fast_approx=use_fast_approximation
)


  0%|          | 0/416 [00:00<?, ?it/s]

Generating graph 0/416...


/Users/mirco/Documents/Tesi/code/models/DGDNN/Data/geometric_dataset_gen.py:133: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df = df[df.index.isin(dates_dt)]
/Users/mirco/Documents/Tesi/code/models/DGDNN/Data/geometric_dataset_gen.py:133: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df = df[df.index.isin(dates_dt)]
/Users/mirco/Documents/Tesi/code/models/DGDNN/Data/geometric_dataset_gen.py:133: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. E

Generating graph 1/416...


/Users/mirco/Documents/Tesi/code/models/DGDNN/Data/geometric_dataset_gen.py:133: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df = df[df.index.isin(dates_dt)]
/Users/mirco/Documents/Tesi/code/models/DGDNN/Data/geometric_dataset_gen.py:133: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df = df[df.index.isin(dates_dt)]
/Users/mirco/Documents/Tesi/code/models/DGDNN/Data/geometric_dataset_gen.py:133: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. E

...

/Users/mirco/Documents/Tesi/code/models/DGDNN/Data/geometric_dataset_gen.py:133: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df = df[df.index.isin(dates_dt)]
/Users/mirco/Documents/Tesi/code/models/DGDNN/Data/geometric_dataset_gen.py:133: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df = df[df.index.isin(dates_dt)]
/Users/mirco/Documents/Tesi/code/models/DGDNN/Data/geometric_dataset_gen.py:133: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. E

Generating graph 414/416...


/Users/mirco/Documents/Tesi/code/models/DGDNN/Data/geometric_dataset_gen.py:133: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df = df[df.index.isin(dates_dt)]
/Users/mirco/Documents/Tesi/code/models/DGDNN/Data/geometric_dataset_gen.py:133: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df = df[df.index.isin(dates_dt)]
/Users/mirco/Documents/Tesi/code/models/DGDNN/Data/geometric_dataset_gen.py:133: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. E

Generating graph 415/416...


/Users/mirco/Documents/Tesi/code/models/DGDNN/Data/geometric_dataset_gen.py:133: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df = df[df.index.isin(dates_dt)]
/Users/mirco/Documents/Tesi/code/models/DGDNN/Data/geometric_dataset_gen.py:133: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df = df[df.index.isin(dates_dt)]
/Users/mirco/Documents/Tesi/code/models/DGDNN/Data/geometric_dataset_gen.py:133: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. E

In [8]:
dataset

# Model 1: GConvGRU

In [ ]:
from torch_geometric_temporal.nn import GConvGRU

/Users/mirco/Documents/Tesi/code/tesi_venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Model 2: DGDNN

GitHub: https://github.com/pixelhero98/DGDNN

In [ ]:
from DGDNN.Model.dgdnn import DGDNN

In [ ]:
model = DGDNN(
    [20],
    [10],
    1,
    1,
    1,
    1,
    1,
    1,
    1
)

sum([p.numel() for p in model.parameters()])

13

## Temp maybe usefull

In [ ]:
from torch_geometric_temporal.signal import DynamicGraphTemporalSignal #Handle dynamic temporal graph